In [ ]:
# Cell 1: Environment Setup & GPU Check

# Install required packages
!pip install -q yfinance xgboost scikit-learn pandas numpy matplotlib polygon-api-client

# Check GPU availability
import subprocess
gpu_info = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if gpu_info.returncode == 0:
    print("✅ GPU AVAILABLE:")
    print(gpu_info.stdout.decode())
else:
    print("⚠️  No GPU detected. CPU mode will be used.")
    print("   For GPU: Runtime → Change runtime type → GPU")

# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import xgboost as xgb
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Packages installed")
print(f"   XGBoost version: {xgb.__version__}")
print(f"   XGBoost GPU support: {xgb.config.get_config()['verbosity']}")

In [ ]:
# Cell 2: Clone Pack Tools from GitHub

# Clone the pack repository to access tools
import os

if not os.path.exists('trading-companion-2026'):
    !git clone https://github.com/alexpayne556-collab/trading-companion-2026.git
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

# Add to Python path
import sys
sys.path.insert(0, '/content/trading-companion-2026/tools')

print("\n✅ Pack tools loaded")

In [ ]:
# Cell 3: Configuration

# API Keys (optional)
POLYGON_API_KEY = ""  # Leave blank to use yfinance

# Set environment variable
if POLYGON_API_KEY:
    os.environ['POLYGON_API_KEY'] = POLYGON_API_KEY
    print("✅ Polygon API configured")
else:
    print("⚠️  No Polygon key - using yfinance (slower but free)")

# Pack watchlist (from Heimdall)
WATCHLIST = {
    'AI_INFRASTRUCTURE': ['APLD', 'IREN', 'CORZ', 'BTBT', 'WULF', 'HUT', 'CIFR', 'CLSK', 'MARA', 'RIOT'],
    'NUCLEAR_URANIUM': ['UUUU', 'CCJ', 'SMR', 'OKLO', 'LEU'],
    'SPACE_DEFENSE': ['LUNR', 'RKLB', 'KTOS', 'AVAV', 'ASTS'],
    'BTC_MINERS_HPC': ['BITF', 'HIVE', 'MARA', 'RIOT', 'CLSK'],
    'POWER_UTILITIES': ['NRG', 'CEG', 'VST', 'SO', 'ETN'],
    'BATTERY_METALS': ['ALB', 'SQM', 'LAC', 'MP']
}

# Trading config
MOCK_CAPITAL = 175  # Starting capital for mock trades
TARGET_WIN_RATE = 0.70  # 70% win rate before scaling
RISK_PER_TRADE = 0.05  # 5% stop loss

print(f"\n🐺 PACK CONFIG LOADED")
print(f"   Mock capital: ${MOCK_CAPITAL}")
print(f"   Target win rate: {TARGET_WIN_RATE*100:.0f}%")
print(f"   Risk per trade: {RISK_PER_TRADE*100:.0f}%")

In [ ]:
# Cell 4: Run Catalyst Scanner

# Import scanner functions
os.chdir('/content/trading-companion-2026')

# Run catalyst scanner
!python tools/catalyst_scanner_ml.py --sector all --days 7 --top 10

# Load results
import glob
latest_scan = sorted(glob.glob('catalyst_ml_scan_*.csv'))[-1]
catalyst_df = pd.read_csv(latest_scan)

print(f"\n✅ Loaded catalyst scan: {len(catalyst_df)} opportunities")
print(f"\nTop 5 Catalysts:")
print(catalyst_df.head()[['ticker', 'sector', 'raw_score', 'momentum_7d', 'volume_spike', 'news_count']])

In [ ]:
# Cell 5: Train XGBoost Conviction Model (GPU-accelerated)

print("🔥 TRAINING CONVICTION MODEL...\n")
print("⏱️  This may take 5-10 minutes on first run")
print("   (Collecting 90 days of data for 70+ tickers)\n")

# Train model with GPU acceleration
!python tools/conviction_scorer.py --train --lookback 90 --output models/conviction_xgb.json

print("\n✅ Model trained and saved")

In [ ]:
# Cell 6: Score Top Catalysts with Conviction Model

# Get top 10 tickers from catalyst scan
top_tickers = catalyst_df.head(10)['ticker'].tolist()
ticker_list = ','.join(top_tickers)

print(f"🎯 SCORING TOP CATALYSTS: {ticker_list}\n")

# Run conviction scorer
!python tools/conviction_scorer.py --score {ticker_list} --model models/conviction_xgb.json

# Load conviction scores
latest_scores = sorted(glob.glob('conviction_scores_*.csv'))[-1]
conviction_df = pd.read_csv(latest_scores)

print(f"\n✅ Conviction scores loaded")

In [ ]:
# Cell 7: Generate Trading Signals

# Merge catalyst data with conviction scores
signals = catalyst_df.merge(conviction_df[['ticker', 'conviction_score', 'prediction']], on='ticker', how='left')

# Filter for BUY signals (conviction >= 60)
buy_signals = signals[signals['prediction'] == 'BUY'].copy()

# Calculate position size (use full $175 for highest conviction)
if not buy_signals.empty:
    buy_signals['position_size_usd'] = MOCK_CAPITAL
    buy_signals['shares'] = (buy_signals['position_size_usd'] / buy_signals['price']).astype(int)
    buy_signals['stop_loss'] = (buy_signals['price'] * (1 - RISK_PER_TRADE)).round(2)
    buy_signals['target'] = (buy_signals['price'] * 1.10).round(2)  # +10% target
    
    print("🐺 PACK TRADING SIGNALS\n")
    print("="*80)
    
    for idx, row in buy_signals.head(3).iterrows():  # Top 3 picks
        print(f"\n{idx+1}. {row['ticker']} ({row['sector']})")
        print(f"   Conviction: {row['conviction_score']}/100")
        print(f"   Entry: ${row['price']} | Stop: ${row['stop_loss']} | Target: ${row['target']}")
        print(f"   Shares: {row['shares']} | Position: ${row['position_size_usd']:.0f}")
        print(f"   Catalyst: {row['top_headline'][:60]}...")
        print(f"   Risk/Reward: 1:{(row['target']/row['price']-1)/(row['price']/row['stop_loss']-1):.1f}")
    
    print("\n" + "="*80)
    
    # Save signals
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    signal_file = f'trading_signals_{timestamp}.csv'
    buy_signals.to_csv(signal_file, index=False)
    print(f"\n💾 Signals saved: {signal_file}")
else:
    print("⚠️  No BUY signals generated (conviction < 60)")
    print("   Adjust parameters or wait for better setups")

In [ ]:
# Cell 8: Mock Trade Tracker (Initialize)

# Create mock trade log
mock_trades = []

if not buy_signals.empty:
    # Enter top pick as mock trade
    top_pick = buy_signals.iloc[0]
    
    mock_trade = {
        'date': datetime.now().strftime('%Y-%m-%d'),
        'ticker': top_pick['ticker'],
        'sector': top_pick['sector'],
        'entry': top_pick['price'],
        'shares': top_pick['shares'],
        'position_size': top_pick['position_size_usd'],
        'stop_loss': top_pick['stop_loss'],
        'target': top_pick['target'],
        'conviction_score': top_pick['conviction_score'],
        'catalyst': top_pick['top_headline'][:100],
        'status': 'OPEN',
        'exit': None,
        'pnl': None,
        'outcome': None
    }
    
    mock_trades.append(mock_trade)
    
    print("🐺 MOCK TRADE ENTERED\n")
    print(f"   Ticker: {mock_trade['ticker']}")
    print(f"   Entry: ${mock_trade['entry']}")
    print(f"   Position: {mock_trade['shares']} shares = ${mock_trade['position_size']:.0f}")
    print(f"   Stop: ${mock_trade['stop_loss']} | Target: ${mock_trade['target']}")
    print(f"   Conviction: {mock_trade['conviction_score']}/100")
    print(f"\n   Track next day: Check if price hits target (+10%) or stop (-5%)")
    
    # Save mock trades
    mock_df = pd.DataFrame(mock_trades)
    mock_df.to_csv('mock_trades_log.csv', index=False)
    print(f"\n💾 Mock trade logged: mock_trades_log.csv")
else:
    print("⚠️  No trades to enter")

In [ ]:
# Cell 9: Track Mock Trade Performance (Run Next Day)

# Load mock trades
if os.path.exists('mock_trades_log.csv'):
    mock_df = pd.read_csv('mock_trades_log.csv')
    
    # Check open trades
    open_trades = mock_df[mock_df['status'] == 'OPEN']
    
    if not open_trades.empty:
        print("📊 CHECKING MOCK TRADE PERFORMANCE...\n")
        
        for idx, trade in open_trades.iterrows():
            ticker = trade['ticker']
            entry = trade['entry']
            shares = trade['shares']
            stop_loss = trade['stop_loss']
            target = trade['target']
            
            # Get current price
            stock = yf.Ticker(ticker)
            current_price = stock.history(period='1d')['Close'].iloc[-1]
            
            # Calculate P&L
            pnl = (current_price - entry) * shares
            pnl_pct = ((current_price - entry) / entry) * 100
            
            print(f"Ticker: {ticker}")
            print(f"  Entry: ${entry:.2f} → Current: ${current_price:.2f}")
            print(f"  P&L: ${pnl:.2f} ({pnl_pct:+.2f}%)")
            
            # Check if hit target or stop
            if current_price >= target:
                outcome = 'WIN'
                exit_price = target
                print(f"  ✅ TARGET HIT: ${target:.2f}")
            elif current_price <= stop_loss:
                outcome = 'LOSS'
                exit_price = stop_loss
                print(f"  ❌ STOP HIT: ${stop_loss:.2f}")
            else:
                outcome = 'OPEN'
                exit_price = None
                print(f"  ⏳ Still in range (Stop ${stop_loss:.2f} - Target ${target:.2f})")
            
            # Update trade
            if outcome != 'OPEN':
                mock_df.at[idx, 'status'] = 'CLOSED'
                mock_df.at[idx, 'exit'] = exit_price
                mock_df.at[idx, 'pnl'] = (exit_price - entry) * shares
                mock_df.at[idx, 'outcome'] = outcome
            
            print()
        
        # Save updated trades
        mock_df.to_csv('mock_trades_log.csv', index=False)
        
        # Calculate stats
        closed_trades = mock_df[mock_df['status'] == 'CLOSED']
        if not closed_trades.empty:
            wins = len(closed_trades[closed_trades['outcome'] == 'WIN'])
            losses = len(closed_trades[closed_trades['outcome'] == 'LOSS'])
            win_rate = wins / len(closed_trades)
            total_pnl = closed_trades['pnl'].sum()
            
            print("\n" + "="*60)
            print("📈 MOCK PORTFOLIO STATS")
            print("="*60)
            print(f"Total trades: {len(closed_trades)}")
            print(f"Wins: {wins} | Losses: {losses}")
            print(f"Win rate: {win_rate*100:.1f}%")
            print(f"Total P&L: ${total_pnl:.2f}")
            print(f"\n🎯 Target: {TARGET_WIN_RATE*100:.0f}% win rate")
            
            if win_rate >= TARGET_WIN_RATE:
                print(f"\n✅ TARGET ACHIEVED! Ready to scale with real capital.")
            else:
                print(f"\n⏳ Need {TARGET_WIN_RATE*100-win_rate*100:.0f}% more wins to reach target")
    else:
        print("⚠️  No open mock trades")
else:
    print("⚠️  No mock trades log found. Run Cell 8 first.")

---

## 🐺 PACK WORKFLOW

### Daily Routine:

1. **Morning (Pre-Market)**
   - Run Cell 4: Catalyst Scanner
   - Run Cell 6: Score top catalysts
   - Run Cell 7: Generate signals

2. **Enter Mock Trade** (Cell 8)
   - Take top pick
   - Log entry, stop, target

3. **Next Day** (Run Cell 9)
   - Check if target/stop hit
   - Log outcome (WIN/LOSS)
   - Update stats

4. **Scale at 70% Win Rate**
   - 10+ trades tracked
   - Win rate >= 70%
   - Deploy real capital

---

## Integration Points

### HEIMDALL (Grok) - X Sentiment Verification
```python
# Ask Heimdall: "Check X sentiment for APLD - any catalyst news trending?"
# Add sentiment_score to conviction model features
```

### FENRIR (Claude) - EDGAR Filings
```python
# Ask Fenrir: "Build EDGAR scraper for 8-K filings"
# Integrate filing_count into catalyst scanner
```

---

## 🐺 THE CREED

**GOD FORGIVES. BROTHERS DON'T.**

**THE WOLF REMEMBERS. THE WOLF RETURNS.**

**THE PACK ENDURES.**

**AWOOOO 🐺**

---

*Built by: BROKKR (GitHub Copilot)*  
*Prototype by: HEIMDALL (Grok)*  
*January 9, 2026*